In [2]:
from google.colab import drive
import os
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [3]:
os.chdir('/content/drive/MyDrive/cs_668')

In [4]:
!pwd

/content/drive/MyDrive/cs_668


Dataets:
  * From LR : iris , from sklearn.datasets
  * From LR :  wine,   from sklearn.datasets
  * From LR: Concrete Compressive Strength
 https://archive.ics.uci.edu/dataset/165/concrete+compressive+strength , uploaded in drive

  * From LR : Breast Cancer Wisconsin : https://www.kaggle.com/datasets/uciml/breast-cancer-wisconsin-data/data , uploaded in drive


In [5]:
import numpy as np
from sklearn.datasets import load_iris,load_wine
import random
from collections import deque
import time
import pandas as pd
from datetime import datetime
import math
from sklearn.metrics import silhouette_score
import warnings
warnings.filterwarnings("ignore")


random.seed(1234)

def euclidean_distance(point1, point2):
    return np.linalg.norm(point1 - point2)

def calculate_sse(clusters, centroids):
    sse = 0
    for i, cluster in enumerate(clusters):
        centroid = centroids[i]
        for point in cluster:
            sse += euclidean_distance(point, centroid) ** 2
    return sse


'''def calculate_sse(clusters, centroids):
    sse = 0
    for i, cluster in enumerate(clusters):
        centroid = centroids[i]
        for point in cluster:
            sse += euclidean_distance(point, centroid) ** 2
        # Penalize for overlap
        for j, other_cluster in enumerate(clusters):
            if i != j:
                for point in other_cluster:
                    sse += euclidean_distance(point, centroid) ** 2
    return sse'''

def calculate_sse_single_cluster(clusters, centroid):
    sse = 0
    for point in cluster:
            sse += euclidean_distance(point, centroid) ** 2
    return sse

def initialize_clusters(data, k):
    n = len(data)
    indices = np.random.choice(n, k, replace=False)
    centroids = data[indices]
    clusters = [[] for _ in range(k)]
    for point in data:
        distances = [euclidean_distance(point, centroid) for centroid in centroids]
        nearest_centroid = np.argmin(distances)
        clusters[nearest_centroid].append(point)
    return clusters, centroids


def is_feasible_move(data_point, current_cluster, new_cluster,k):
    return len(new_cluster) < k and data_point not in new_cluster


**Methods**

Population based metaheuristics:


*   Genetic Algorithm




In [6]:
import numpy as np
#from sklearn.datasets import load_breast_cancer
from sklearn.metrics import silhouette_score


# centriod of clusters
def get_centroids(clusters):
   centroids = []
   for cluster in clusters:
      centroid = np.mean(cluster, axis=0)
      centroids.append(centroid)
   return centroids

# clustering initialization
def initialize_clusters(data, k):
    n = len(data)
    indices = np.random.choice(n, k, replace=False)
    centroids = data[indices]
    clusters = [[] for _ in range(k)]

    for point in data:
        distances = [euclidean_distance(point, centroid)
                       for centroid in centroids]
        nearest_centroid = np.argmin(distances)
        clusters[nearest_centroid].append(point)

    return clusters, centroids

# distance calculation
def euclidean_distance(point1, point2):
    return np.linalg.norm(point1 - point2)

# calculate_sse
def calculate_sse(clusters, centroids):
    sse = 0
    for i, cluster in enumerate(clusters):
        centroid = centroids[i]
        for point in cluster:
            sse += euclidean_distance(point, centroid)**2
    return sse

# Decode chromosome to clusters
def decode(solution):
   clusters = [[] for _ in range(num_clusters)]
   for i, cluster in enumerate(solution):
       clusters[cluster].append(X[i])
   return clusters


# Fitness function
def fitness_function(solution):
   clusters = decode(solution)
   centroids = get_centroids(clusters)
   sse = calculate_sse(clusters, centroids)
   return 1/sse


# population initialization
def initialize_population():
   population = []
   for _ in range(population_size):
       solution = np.random.choice([0, 1], size=X.shape[1], replace=True)
       population.append(solution)

   return population

# crossover
def crossover(parent1, parent2):
   crossover_point = np.random.randint(1, len(parent1) - 1)
   child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
   child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
   return child1, child2

# mutation
def mutate(solution):
   mutated_solution = np.copy(solution)
   for i in range(len(solution)):
       if np.random.random() < mutation_rate:
           mutated_solution[i] = 1 - mutated_solution[i]
   return mutated_solution

# selection
def select_parents(population, fitness_values):
   total_fitness = np.sum(fitness_values)
   probabilities = fitness_values / total_fitness
   selected_indices = np.random.choice(range(len(population)), size=2, replace=False, p=probabilities)
   return population[selected_indices[0]], population[selected_indices[1]]

# Evaluation
def evaluate_population(population):
   fitness_values = []
   for solution in population:
       fitness = fitness_function(solution)
       fitness_values.append(fitness)
   return fitness_values

# Main GA loop
def ga_clustering():

   start_time = time.time()

   population = initialize_population()
   for _ in range(population_size):
      fitness_values = evaluate_population(population)
      new_population = []

      for _ in range(population_size // 2):
          parent1, parent2 = select_parents(population, fitness_values)
          child1, child2 = crossover(parent1, parent2)
          mutated_child1 = mutate(child1)
          mutated_child2 = mutate(child2)
          new_population.extend([mutated_child1, mutated_child2])

      population = new_population

   best_solution = population[np.argmax(fitness_values)]
   end_time = time.time()
   execution_time = end_time - start_time

   # Evaluate best solution
   clusters = decode(best_solution)
   centroids = get_centroids(clusters)
   sse = calculate_sse(clusters, centroids)

   # to compute silhouette_score we have to get labels based on clusters
   labels = [-1] * len(X)
   for i, cluster in enumerate(clusters):
      for j, point in enumerate(cluster):
        labels[j] = i

   silhouette = silhouette_score(X, labels)

   return best_solution, sse, silhouette,execution_time

In [ ]:
result=[]

In [ ]:

def data_loader(choice, cases):

    data = num_cluster = 0
    match choice:
        case 1:

            data = load_iris().data
            num_cluster = len(set(load_iris().target))

        case 2:

            data = load_wine().data
            num_cluster = len(set(load_wine().target))

        case 3:

            file_ = pd.read_excel('concrete_data.xlsx')
            data = file_.iloc[:, :-1].values
            num_cluster = 7

        case 4:

            file_= pd.read_csv('breast_cancer_data.csv')
            data = file_.drop(columns=['diagnosis']).values
            num_cluster = file_['diagnosis'].nunique()

        case _:
            print("no data has been choosen")
    return data, num_cluster


datasets_names={1: "iris_data", 2: "wine_data", 3: "concrete_data", 4: "breast_cancer_data" }



In [7]:


population_size_list = [30,50,100]

mutation_rate_list = [0.001,0.01,0.1]


Data_choice = [1,2,3,4]

number_of_runs = 10
sse = execution_time = 0

In [ ]:
for i in range(number_of_runs):
  for j in Data_choice:
    for p in population_size_list:
      for m in mutation_rate_list:
        dataset_name = datasets_names.get(j)
        X , num_clusters = data_loader(j, datasets_names)
        population_size = p
        mutation_rate = m

        best_solution, sse, silhouette,execution_time = ga_clustering()

        result.append({
                    "dataset": dataset_name,
                    "algorithm": 'GA',
                    "best_sse": sse,
                    "execution_time": execution_time,
                    "silhouette_avg": silhouette,
                    "population_size":population_size,
                    "mutation_rate": mutation_rate,
                    "run_no": i
                })

      #print("Done with Dataset", dataset_name)
  print("Done with run no:", i)


Done with run no: 0
Done with run no: 1
Done with run no: 2
Done with run no: 3
Done with run no: 4
Done with run no: 5
Done with run no: 6
Done with run no: 7
Done with run no: 8
Done with run no: 9


In [8]:
#result

In [ ]:

current_datetime = datetime.now()
filename = current_datetime.strftime("%Y-%m-%d_%H-%M-%S") + ".xlsx"
final_result = pd.DataFrame(result)
final_result.to_excel('result_'+filename)